In [64]:
import pandas as pd
import numpy as np

from datetime import datetime

# 훈련 데이터 전처리

In [65]:
df = pd.read_excel('C:/MachineLearning/Resource/Data_Train.xlsx')
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [66]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [67]:
df.drop(columns=['Route'],inplace=True)

In [68]:
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format= '%H:%M').dt.time
df['Duration_hour'] = df.Duration.str.extract('(\d+)h')
df['Duration_min'] = df.Duration.str.extract('(\d+)m').fillna(0)
df.drop('Duration', axis=1, inplace=True)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Duration_hour,Duration_min
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20:00,01:10 22 Mar,non-stop,No info,3897,2,50
1,Air India,1/05/2019,Kolkata,Banglore,05:50:00,13:15,2 stops,No info,7662,7,25
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25:00,04:25 10 Jun,2 stops,No info,13882,19,0
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05:00,23:30,1 stop,No info,6218,5,25
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50:00,21:35,1 stop,No info,13302,4,45


# Duration 에서 이상치 drop

In [69]:
df[df['Duration_hour'].isna()]
df.drop(index=6474,inplace=True)
len(df)

10682

In [70]:
# str -> int64 로 변경
df.Duration_hour = df.Duration_hour.astype('int64')
df.Duration_min = df.Duration_min.astype('int64')

In [71]:
# Duration_hour을 분으로 변경
df.Duration_hour = df.Duration_hour*60
df['Duration_total'] = df.Duration_hour+df.Duration_min
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Duration_hour,Duration_min,Duration_total
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20:00,01:10 22 Mar,non-stop,No info,3897,120,50,170
1,Air India,1/05/2019,Kolkata,Banglore,05:50:00,13:15,2 stops,No info,7662,420,25,445
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25:00,04:25 10 Jun,2 stops,No info,13882,1140,0,1140
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05:00,23:30,1 stop,No info,6218,300,25,325
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50:00,21:35,1 stop,No info,13302,240,45,285


In [72]:
df.drop(columns=['Duration_hour','Duration_min','Arrival_Time'],inplace=True)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20:00,non-stop,No info,3897,170
1,Air India,1/05/2019,Kolkata,Banglore,05:50:00,2 stops,No info,7662,445
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25:00,2 stops,No info,13882,1140
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05:00,1 stop,No info,6218,325
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50:00,1 stop,No info,13302,285


# Airline 전처리

In [73]:
# count값에 맞춰 index 넘버링하기
air_count = df.Airline.value_counts().index

# 200 보다 적은 수의 airline은 Others 로 변환
airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
df.Airline = df.Airline.replace(airlist, 'Others')
df.Airline.value_counts()

# 컬럼 미리 생성
df['Air_col'] = 0

df.Airline.value_counts()

Jet Airways          3849
IndiGo               2053
Air India            1751
Multiple carriers    1196
SpiceJet              818
Vistara               479
Air Asia              319
Others                217
Name: Airline, dtype: int64

In [74]:
# Air_col : Airline을 번호로 분류
for t in range(len(air_count)):
    df.loc[df.Airline == air_count[t], 'Air_col'] = t

print(df.Air_col.value_counts())
print(df.head())

0    4066
1    2053
2    1751
3    1196
4     818
5     479
6     319
Name: Air_col, dtype: int64
       Airline Date_of_Journey    Source Destination  Dep_Time Total_Stops  \
0       IndiGo      24/03/2019  Banglore   New Delhi  22:20:00    non-stop   
1    Air India       1/05/2019   Kolkata    Banglore  05:50:00     2 stops   
2  Jet Airways       9/06/2019     Delhi      Cochin  09:25:00     2 stops   
3       IndiGo      12/05/2019   Kolkata    Banglore  18:05:00      1 stop   
4       IndiGo      01/03/2019  Banglore   New Delhi  16:50:00      1 stop   

  Additional_Info  Price  Duration_total  Air_col  
0         No info   3897             170        1  
1         No info   7662             445        2  
2         No info  13882            1140        0  
3         No info   6218             325        1  
4         No info  13302             285        1  


In [75]:
df.drop(columns=['Airline'],inplace=True)
df.head()

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col
0,24/03/2019,Banglore,New Delhi,22:20:00,non-stop,No info,3897,170,1
1,1/05/2019,Kolkata,Banglore,05:50:00,2 stops,No info,7662,445,2
2,9/06/2019,Delhi,Cochin,09:25:00,2 stops,No info,13882,1140,0
3,12/05/2019,Kolkata,Banglore,18:05:00,1 stop,No info,6218,325,1
4,01/03/2019,Banglore,New Delhi,16:50:00,1 stop,No info,13302,285,1


# Additional_Info 전처리

In [76]:
# 3. Additional_Info 전처리
# 20 보다 적은 수의 Additional_Info Others 로 변환
add_count = df.Additional_Info.value_counts().index
additional_thing = [l for l in add_count if list(df.Additional_Info).count(l) < 20]
df.Additional_Info = df.Additional_Info.replace(additional_thing, 'Others')
df.Additional_Info.value_counts()

No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
Others                            36
Name: Additional_Info, dtype: int64

In [77]:
# 컬럼 미리 생성
df['Add_col'] = 0
df.head()

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col
0,24/03/2019,Banglore,New Delhi,22:20:00,non-stop,No info,3897,170,1,0
1,1/05/2019,Kolkata,Banglore,05:50:00,2 stops,No info,7662,445,2,0
2,9/06/2019,Delhi,Cochin,09:25:00,2 stops,No info,13882,1140,0,0
3,12/05/2019,Kolkata,Banglore,18:05:00,1 stop,No info,6218,325,1,0
4,01/03/2019,Banglore,New Delhi,16:50:00,1 stop,No info,13302,285,1,0


In [78]:
add_count = df.Additional_Info.value_counts().index
add_count

Index(['No info', 'In-flight meal not included',
       'No check-in baggage included', 'Others'],
      dtype='object')

In [79]:
# Add_col 컬럼에 인덱스 번호로 넣기
for t in range(len(add_count)):
    df.loc[df.Additional_Info == add_count[t], 'Add_col'] = t

df.Add_col.value_counts()

0    8344
1    1982
2     320
3      36
Name: Add_col, dtype: int64

In [80]:
df.head()

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col
0,24/03/2019,Banglore,New Delhi,22:20:00,non-stop,No info,3897,170,1,0
1,1/05/2019,Kolkata,Banglore,05:50:00,2 stops,No info,7662,445,2,0
2,9/06/2019,Delhi,Cochin,09:25:00,2 stops,No info,13882,1140,0,0
3,12/05/2019,Kolkata,Banglore,18:05:00,1 stop,No info,6218,325,1,0
4,01/03/2019,Banglore,New Delhi,16:50:00,1 stop,No info,13302,285,1,0


# Total_Stops 전처리

In [81]:
# 결론 : 전반적으로 1 stop이 절반 이상이므로 1 stop 으로 처리
df.loc[df.Total_Stops.isna(),'Total_Stops'] = '1 stop'
df[df.Total_Stops.isna()]

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col


In [82]:
# 각 stop에 맞춘 
def handle_stops(x):
    # if type(x) == float: return np.NaN
    if x == 'non-stop': return 0
    return int(x.split()[0])

df.Total_Stops.apply(handle_stops)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10682, dtype: int64

In [83]:
df.Total_Stops = df.Total_Stops.apply(handle_stops)
df.head()

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col
0,24/03/2019,Banglore,New Delhi,22:20:00,0,No info,3897,170,1,0
1,1/05/2019,Kolkata,Banglore,05:50:00,2,No info,7662,445,2,0
2,9/06/2019,Delhi,Cochin,09:25:00,2,No info,13882,1140,0,0
3,12/05/2019,Kolkata,Banglore,18:05:00,1,No info,6218,325,1,0
4,01/03/2019,Banglore,New Delhi,16:50:00,1,No info,13302,285,1,0


# Date_of_Journey

In [84]:
import datetime

days = ['MON', 'TUE', 'WEN', 'THU', 'FRI', 'SAT', 'SUN']

In [85]:
k = df.Date_of_Journey.str.extract('(\d+)/')
df['DOJ'] = k
df.head()

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col,DOJ
0,24/03/2019,Banglore,New Delhi,22:20:00,0,No info,3897,170,1,0,24
1,1/05/2019,Kolkata,Banglore,05:50:00,2,No info,7662,445,2,0,1
2,9/06/2019,Delhi,Cochin,09:25:00,2,No info,13882,1140,0,0,9
3,12/05/2019,Kolkata,Banglore,18:05:00,1,No info,6218,325,1,0,12
4,01/03/2019,Banglore,New Delhi,16:50:00,1,No info,13302,285,1,0,01


In [86]:
df['Date_of_journey_DT'] = pd.to_datetime(df['Date_of_Journey'])

In [89]:
df['weekday'] = pd.to_datetime(df['Date_of_journey_DT']).dt.weekday
df

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col,DOJ,Date_of_journey_DT,weekday
0,24/03/2019,Banglore,New Delhi,22:20:00,0,No info,3897,170,1,0,24,2019-03-24,6
1,1/05/2019,Kolkata,Banglore,05:50:00,2,No info,7662,445,2,0,1,2019-01-05,5
2,9/06/2019,Delhi,Cochin,09:25:00,2,No info,13882,1140,0,0,9,2019-09-06,4
3,12/05/2019,Kolkata,Banglore,18:05:00,1,No info,6218,325,1,0,12,2019-12-05,3
4,01/03/2019,Banglore,New Delhi,16:50:00,1,No info,13302,285,1,0,01,2019-01-03,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,9/04/2019,Kolkata,Banglore,19:55:00,0,No info,4107,150,6,0,9,2019-09-04,2
10679,27/04/2019,Kolkata,Banglore,20:45:00,0,No info,4145,155,2,0,27,2019-04-27,5
10680,27/04/2019,Banglore,Delhi,08:20:00,0,No info,7229,180,0,0,27,2019-04-27,5
10681,01/03/2019,Banglore,New Delhi,11:30:00,0,No info,12648,160,5,0,01,2019-01-03,3


In [92]:
df['weekday_name'] = pd.to_datetime(df['Date_of_journey_DT']).dt.day_name()
df

,Date_of_Journey,Source,Destination,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col,DOJ,Date_of_journey_DT,weekday,weekday_name
0,24/03/2019,Banglore,New Delhi,22:20:00,0,No info,3897,170,1,0,24,2019-03-24,6,Sunday
1,1/05/2019,Kolkata,Banglore,05:50:00,2,No info,7662,445,2,0,1,2019-01-05,5,Saturday
2,9/06/2019,Delhi,Cochin,09:25:00,2,No info,13882,1140,0,0,9,2019-09-06,4,Friday
3,12/05/2019,Kolkata,Banglore,18:05:00,1,No info,6218,325,1,0,12,2019-12-05,3,Thursday
4,01/03/2019,Banglore,New Delhi,16:50:00,1,No info,13302,285,1,0,01,2019-01-03,3,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,9/04/2019,Kolkata,Banglore,19:55:00,0,No info,4107,150,6,0,9,2019-09-04,2,Wednesday
10679,27/04/2019,Kolkata,Banglore,20:45:00,0,No info,4145,155,2,0,27,2019-04-27,5,Saturday
10680,27/04/2019,Banglore,Delhi,08:20:00,0,No info,7229,180,0,0,27,2019-04-27,5,Saturday
10681,01/03/2019,Banglore,New Delhi,11:30:00,0,No info,12648,160,5,0,01,2019-01-03,3,Thursday


In [87]:
def chg_date(txt):
  return days[int(txt.split('/')[0])]

df.Date_of_Journey = df.apply(chg_date)
df.head()

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
df.drop(columns=['Source','Destination'],inplace=True)
df.head()

,Date_of_Journey,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col
0,THU,22:20:00,0,No info,3897,170,1,0
1,SAT,05:50:00,2,No info,7662,445,2,0
2,SUN,09:25:00,2,No info,13882,1140,0,0
3,SAT,18:05:00,1,No info,6218,325,1,0
4,THU,16:50:00,1,No info,13302,285,1,0


In [ ]:
df.Dep_Time = df.Dep_Time.astype(str)

In [ ]:
# 출발시간, 도착시간에서 시간 단위만 뽑기
df['Dep_hour'] = df.Dep_Time.str.extract('([0-9]+)\:')
# df['Arrival_hour'] = df.Arrival_Time.str.extract('([0-9]+)\:')


In [ ]:
df.head()

,Date_of_Journey,Dep_Time,Total_Stops,Additional_Info,Price,Duration_total,Air_col,Add_col,Dep_hour
0,THU,22:20:00,0,No info,3897,170,1,0,22
1,SAT,05:50:00,2,No info,7662,445,2,0,05
2,SUN,09:25:00,2,No info,13882,1140,0,0,09
3,SAT,18:05:00,1,No info,6218,325,1,0,18
4,THU,16:50:00,1,No info,13302,285,1,0,16


In [ ]:
df.drop(columns=['Additional_Info','Dep_Time'],inplace=True)
df.head()

,Date_of_Journey,Total_Stops,Price,Duration_total,Air_col,Add_col,Dep_hour
0,THU,0,3897,170,1,0,22
1,SAT,2,7662,445,2,0,05
2,SUN,2,13882,1140,0,0,09
3,SAT,1,6218,325,1,0,18
4,THU,1,13302,285,1,0,16


In [ ]:
# 범주형 변수 처리
df = pd.get_dummies(df, columns=['Date_of_Journey'],drop_first=True)
df.head()

,Total_Stops,Price,Duration_total,Air_col,Add_col,Dep_hour,Date_of_Journey_SAT,Date_of_Journey_SUN,Date_of_Journey_THU
0,0,3897,170,1,0,22,0,0,1
1,2,7662,445,2,0,05,1,0,0
2,2,13882,1140,0,0,09,0,1,0
3,1,6218,325,1,0,18,1,0,0
4,1,13302,285,1,0,16,0,0,1
